In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result for CSV files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. The data is stored in CSV format. The path to the CSV files is specified in the `graphrag/test_csvs/Lancedbai_Full_Employees_GraphRAG_With_DOB.csv` variable.

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](./graphrag-neo4j-csv-visualization.PNG)

# How to configure the settings.yml File

```yaml
input:
  type: file # Use 'blob' if processing data from blob storage
  file_type: csv
  base_dir: "input"
  file_encoding: "utf-8"
  file_pattern: ".*\\.csv$"
  source_column: "source"            
  text_column: "text"
  timestamp_column: "timestamp"
  timestamp_format: "%Y-%m-%d"
```

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [ ]:
GRAPHRAG_FOLDER = "PATH_TO_OUTPUT/artifacts"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [ ]:
%pip install --quiet pandas neo4j-rust-ext

In [4]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [ ]:
NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""  # your password
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [6]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`

In [7]:
# create constraints, idempotent operation

statements = [
    "\ncreate constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique",
    "\ncreate constraint document_id if not exists for (d:__Document__) require d.id is unique",
    "\ncreate constraint entity_id if not exists for (c:__Community__) require c.community is unique",
    "\ncreate constraint entity_id if not exists for (e:__Entity__) require e.id is unique",
    "\ncreate constraint entity_title if not exists for (e:__Entity__) require e.name is unique",
    "\ncreate constraint entity_title if not exists for (e:__Covariate__) require e.title is unique",
    "\ncreate constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique",
    "\n",
]

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

For `create_final_documents.parquet`:
- Text format input uses `title` column
- CSV format input uses `text` column

This means when importing documents:
- Text format parquet file: reads and processes the `title` column
- CSV format parquet file: reads and processes the `text` column

In [8]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "text"]
)
doc_df.head(2)

,id,text
0,11178d482ef61bf651611ed9f3a6c75a1bb166ad98edb9...,"Name: Ishita Mann, Designation: Automation Eng..."
1,8479222f1f8933d7367006efb37a7af8753754c4c0dc29...,"Name: Riya Mallick, Designation: AI Engineer, ..."


In [ ]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.text}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 60, 'nodes_created': 60, 'properties_set': 120}
60 rows in 0.03634047508239746 s.


60

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [10]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,913f0820a9755495a456b3f9d25df7fa1da5628534d89e...,"Name: Ela Sarin, Designation: Software Enginee...",36,[03b6640b26cb709e45cf86bd502fa72037ef7095da26c...
1,7aa39f4da7a467af172480d070a9ffada3e3454e63cb90...,"Name: Inaaya Sengupta, Designation: AI Engine...",39,[0b3df84bc39312f6cc47dc4b653661999eabc1b24534d...


In [11]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 60, 'relationships_created': 60, 'nodes_created': 60, 'properties_set': 180}
60 rows in 0.04514002799987793 s.


60

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

For `create_final_entities.parquet`:
- Text format input uses `name` column 
- CSV format input uses `title` column

This means when importing entities:
- Text format parquet file: reads and processes the `name` column
- CSV format parquet file: reads and processes the `title` column

In [ ]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "title", 
        "type",
        "description",
        "human_readable_id",
        "id",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,title,type,description,human_readable_id,id,text_unit_ids
0,ELA SARIN,PERSON,Ela Sarin is a Software Engineer based in Lond...,0,c1cdc5b3-4cf0-41b9-8133-fe0ae2325961,[913f0820a9755495a456b3f9d25df7fa1da5628534d89...
1,NAYANTARA IYER,PERSON,Nayantara Iyer is a manager who oversees a tea...,1,f556bfa4-f43c-4110-83a9-c760009c37c5,[913f0820a9755495a456b3f9d25df7fa1da5628534d89...


In [ ]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.title,'"','')}
WITH e, value
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 119, 'relationships_created': 279, 'nodes_created': 119, 'properties_set': 476}
119 rows in 0.2919292449951172 s.


119

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [ ]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "source",
        "target",
        "id",
        "weight",
        "human_readable_id",
        "description",
        "text_unit_ids",
    ],
)
rel_df.head(2)

,source,target,id,weight,human_readable_id,description,text_unit_ids
0,ELA SARIN,NAYANTARA IYER,bab4f0e1-ea51-43ba-b2b4-a6ff6f4c290b,9.0,0,Ela Sarin is managed by Nayantara Iyer,[913f0820a9755495a456b3f9d25df7fa1da5628534d89...
1,ELA SARIN,LONDON,50690419-52ea-4f8a-b9ba-d28f08ad3198,1.0,1,Ela Sarin is located in London,[913f0820a9755495a456b3f9d25df7fa1da5628534d89...


In [ ]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 225, 'properties_set': 1125}
225 rows in 0.12598586082458496 s.


225

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [16]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,a0bd8993-7109-45f4-870d-16db412b616a,0,Community 0,[2899252a7de3e801a03a25fc2275f849069be82c97619...,"[15755ab2-5931-4332-a4ef-25ab64bf8431, 2555d64..."
1,6a8d6b53-3e0b-4345-ae27-4223e2a44ccc,0,Community 1,[58985f8c9caa3ac1e72e1401e3f8fe3eaf45e30f14358...,"[2d10758b-096c-468a-956b-41e48f73aa0b, 62c20e7..."


In [17]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 36, 'relationships_created': 221, 'nodes_created': 36, 'properties_set': 108}
36 rows in 0.24303984642028809 s.


36

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [18]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,e897ccf1e1014228b8f7bb067f8c6b2a,18,1,Software Engineers and Their Managers in Londo...,The community consists of software engineers b...,[{'explanation': 'Raghav Barad is a software e...,3.5,"The impact severity rating is low to moderate,...",# Software Engineers and Their Managers in Lon...
1,5fb58eabc6bf4a8a91ebbd1088231f05,21,1,Zain Sahota and Onkar Lata Community,"The community revolves around Zain Sahota, a D...",[{'explanation': 'Zain Sahota is a Data Scient...,2.5,The impact severity rating is low due to the l...,# Zain Sahota and Onkar Lata Community\n\nThe ...


In [19]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 30, 'relationships_created': 26, 'nodes_created': 30, 'properties_set': 106}
4 rows in 0.12617111206054688 s.


4

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag